In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision 
import torchvision.transforms as transforms
from PIL import Image
import os
import random
from customDataset import CustomDataset
from torch.utils.data import DataLoader

from neuralNet import NeuralNet
from VGG import VGGLoss

In [2]:
### Check if GPU is available

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Check for GPU availability and set device
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    device = torch.device("cuda")
    current_device_idx = torch.cuda.current_device()
else:
    device_count = 0
    device = torch.device("cpu")
    current_device_idx = None

# Print device information
print(f"Number of available GPUs: {device_count}")
print(f"Current device index: {current_device_idx}")
# print(f"Current device name: {torch.cuda.get_device_name(current_device_idx)}")

Using CPU
Number of available GPUs: 0
Current device index: None


In [3]:
# Dataset creation
# hyperparameters
batch_size = 1
train_X = "LR_train"
train_y = "HR_train"
val_X = "LR_val"
val_y = "HR_val"
test_X = "LR_test"
test_y = "HR_test"


# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Define a transformation operation to normalize the images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # normalize to [-1, 1]
])


train_dataset = CustomDataset(train_X, train_y, transform=transforms.ToTensor())
val_dataset = CustomDataset(val_X, val_y, transform=transforms.ToTensor())
test_dataset = CustomDataset(test_X, test_y, transform=transforms.ToTensor())

print("train dataset size: ", len(train_dataset))
print("val dataset size: ", len(val_dataset))
print("test dataset size: ", len(test_dataset))

#### Create the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

train dataset size:  100
val dataset size:  50
test dataset size:  20


In [4]:
#### Create the model
model = NeuralNet()

# create optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# move them to device (GPU/CPU)
model.to(device)

# loss function
loss_fn1 = VGGLoss(device)
loss_fn2 = nn.MSELoss()

epochs = 10

In [5]:
#### Training loop
for epoch in range(epochs):
    print(f"------------------\n Epoch: {epoch}")
    model.train()
    epoch_loss = 0
    batch_loss = 0
    for i, (X, y) in enumerate(train_dataloader):
        if i % 100 == 0:
            print(" Batch:", i, " Loss:", batch_loss/100)
            batch_loss = 0
        
        # retrieve data and move tensors to device
        inputs = X.to(device)
        labels = y.to(device)
        
        # calculate loss
        outputs = model(inputs)
        loss = loss_fn1(outputs, labels)
        loss+= loss_fn2(outputs, labels)
        epoch_loss += loss.item()
        batch_loss += loss.item()
        
        # zero gradient and backpropagate
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # change learning rate
        if epoch/epochs > 0.5:
            optimizer = optim.Adam(model.parameters(), lr=1e-4)
            
        
    # Print epoch information
    print(f"Epoch {epoch+1}/{epochs}: Loss: {epoch_loss:.4f}")
    

<class 'generator'> Training

------------------
 Epoch: 0
 Batch: 0  Loss: 0.0
Epoch 1/10: Loss: 3229.5569
------------------
 Epoch: 1
 Batch: 0  Loss: 0.0
Epoch 2/10: Loss: 2063.8245
------------------
 Epoch: 2
 Batch: 0  Loss: 0.0
Epoch 3/10: Loss: 1868.1465
------------------
 Epoch: 3
 Batch: 0  Loss: 0.0
Epoch 4/10: Loss: 1641.2900
------------------
 Epoch: 4
 Batch: 0  Loss: 0.0
Epoch 5/10: Loss: 1578.2311
------------------
 Epoch: 5
 Batch: 0  Loss: 0.0
Epoch 6/10: Loss: 1544.1122
------------------
 Epoch: 6
 Batch: 0  Loss: 0.0
Epoch 7/10: Loss: 1411.2665
------------------
 Epoch: 7
 Batch: 0  Loss: 0.0
Epoch 8/10: Loss: 1337.3043
------------------
 Epoch: 8
 Batch: 0  Loss: 0.0
Epoch 9/10: Loss: 1324.5937
------------------
 Epoch: 9
 Batch: 0  Loss: 0.0
Epoch 10/10: Loss: 1321.1975


In [23]:
# Preprocess the input image
pictures = os.listdir("LR_images/ci")
random.shuffle(pictures)
print(pictures[0])
# image = Image.open(os.path.join("LR_images/ci",pictures[0]))
# hr_image = Image.open(os.path.join("HR_images",pictures[0][14:]))
image = Image.open('downscaled_ci_waves.jpg')
hr_image = Image.open('waves.jpg')
image.show()
hr_image.show()
input_image = transform(image)

# # Load the trained model
# model = DeepImagePrior(input_shape)
# model.load_state_dict(torch.load(PATH_TO_SAVED_WEIGHTS))

# Set the model to evaluation mode
model.eval()

# Perform the inference
with torch.no_grad():
    input_image = input_image.to(device)  # Move the input image to the device
    input_image = input_image.unsqueeze(0)  # Add a batch dimension
    output = model(input_image)

# Convert the output to a PIL image
output = output.squeeze(0)
output = transforms.ToPILImage()(output)
output.show()

downscaled_ci_pixabay_dog_000045.jpg


In [6]:
##############################################
### SAVE MODEL, LOAD MODEL, WORKS ANYWHERE ###
##############################################

model_name = "model.pt"

# Save the model
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, 'checkpoint.pth')

# Load the model
new_model = NeuralNet()
checkpoint = torch.load('checkpoint.pth')
new_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

print(loss)

tensor(12.9768, requires_grad=True)


In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

189274